# Notebook to generate charts for the smaller date intervals used in the report.

In [ ]:
# import packages
import numpy as np
import matplotlib as mplt
import matplotlib.pyplot as plt
import datetime as dt
import random as random
import pandas as pd
import seaborn as sns
from scipy.optimize import curve_fit
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
import os
# get current working directory to load and save files
thedir = os.getcwd()
# use cwd to to get folders containing figures and data
datadir = os.path.abspath(os.path.join(os.path.dirname(thedir), '..', 'acrop/model_results'))
figdir = os.path.abspath(os.path.join(os.path.dirname(thedir), '..', 'acrop/figures'))

from scipy.stats import ttest_rel
from scipy.stats import ttest_ind

In [ ]:
# collect the data for graphing by season, aggregating by depth and interval
depths = [5,10,15,20,25]
dates= ["30_60","30_80","30_110","60_110","60_130","80_110","80_140","110_140"]
# load files
dates_s = []
for i in depths:
    for j in dates:
        df = pd.read_csv(F'{datadir}/{j}_{i}s.csv')
        df["depth"] = i
        df["interval"] = j
        dates_s.append(df)
# concatonate the three loaded results into one df.
for i in range(len(dates_s)):
    if i == 0:
        merged_s = pd.DataFrame()
    merged_s = pd.concat([merged_s,dates_s[i]],axis=0)
merged_s.reset_index(inplace=True,drop=True)

In [ ]:
# plot the seasonal data

sns.set_style('darkgrid')
fig, axs = plt.subplots(4,2,figsize=(16,16))
data_= merged_s[merged_s.interval == '30_60']
sns.lineplot(data=data_, x='Season',y='Yield (tonne/ha)_mean',hue = 'depth',ax=axs[0,0])
axs[0,0].set_title('30_60')
data_= merged_s[merged_s.interval == '30_80']
sns.lineplot(data=data_, x='Season',y='Yield (tonne/ha)_mean',hue = 'depth',ax=axs[0,1])
axs[1,0].set_title('30_80')
data_= merged_s[merged_s.interval == '30_110']
sns.lineplot(data=data_, x='Season',y='Yield (tonne/ha)_mean',hue = 'depth',ax=axs[1,0])
axs[0,1].set_title('30_110')
data_= merged_s[merged_s.interval == '60_110']
sns.lineplot(data=data_, x='Season',y='Yield (tonne/ha)_mean',hue = 'depth',ax = axs[1,1])
axs[1,1].set_title('60_110')
data_= merged_s[merged_s.interval == '60_130']
sns.lineplot(data=data_, x='Season',y='Yield (tonne/ha)_mean',hue = 'depth',ax=axs[2,0])
axs[2,0].set_title('60_130')
data_= merged_s[merged_s.interval == '80_110']
sns.lineplot(data=data_, x='Season',y='Yield (tonne/ha)_mean',hue = 'depth',ax=axs[2,1])
axs[2,1].set_title('80_110')
data_= merged_s[merged_s.interval == '80_140']
sns.lineplot(data=data_, x='Season',y='Yield (tonne/ha)_mean',hue = 'depth',ax=axs[3,0])
axs[3,0].set_title('80_140')
data_= merged_s[merged_s.interval == '110_140']
sns.lineplot(data=data_, x='Season',y='Yield (tonne/ha)_mean',hue = 'depth',ax = axs[3,1])
axs[3,1].set_title('110_140')
plt.tight_layout()
plt.savefig(F"{figdir}/season_")

In [7]:
# get the files on the data aggregated by runs for the main analysis.

depths = [5,10,15,20,25]
dates= ["30_60","30_80","30_110","60_110","60_130","80_110","80_140","110_140"]
# load files
dates_ = []
for i in depths:
    for j in dates:
        df = pd.read_csv(F'{datadir}/{j}_{i}.csv')
        df["depth"] = i
        df["interval"] = j
        dates_.append(df)
# concatonate the three loaded results into one df.
for i in range(len(dates_)):
    if i == 0:
        merged = pd.DataFrame()
    merged = pd.concat([merged,dates_[i]],axis=0)

In [ ]:
# get the top n yields and display it
top = merged.nlargest(15,["Yield (tonne/ha)_mean"])
top.reset_index(inplace=True,drop=True)
top = top[["Yield (tonne/ha)_mean",'irrigation_1',
       'irrigation_2', 'depth','interval']]
top

In [9]:
# obtain the mean of each of the variables in the graphs to draw the line plots

data_ = merged
group_mean = data_.groupby(["interval","depth"]).agg({'Yield (tonne/ha)_mean' : 'mean', 
                            'biomass_mean' : 'mean',
                            'harvest_index_mean' : 'mean',
                            'canopy_cover_mean': 'mean',
                            'Es_mean' : 'mean',
                            'Tr_mean': 'mean',
                            'DeepPerc_mean': 'mean',
                            'CR_mean': 'mean'})
#group_mean.columns = ["_".join(pair) for pair in group_mean.columns]
group_mean.reset_index(drop = False,inplace = True)

group_max = data_.groupby(["interval","depth"]).agg({'Yield (tonne/ha)_max' : 'mean', 
                            'biomass_max' : 'mean',
                            'harvest_index_max' : 'mean',
                            'canopy_cover_max': 'mean',
                            'Es_max' : 'mean',
                            'Tr_max': 'mean',
                            'DeepPerc_max': 'mean',
                            'CR_max': 'mean'})
group_max.reset_index(drop = False,inplace = True)
group_min = data_.groupby(["interval","depth"]).agg({'Yield (tonne/ha)_min' : 'mean', 
                            'biomass_min' : 'mean',
                            'harvest_index_min' : 'mean',
                            'canopy_cover_min': 'mean',
                            'Es_min' : 'mean',
                            'Tr_min': 'mean',
                            'DeepPerc_min': 'mean',
                            'CR_min': 'mean'})
group_min.reset_index(drop = False,inplace = True)
group_var = data_.groupby(["interval","depth"]).agg({'Yield (tonne/ha)_var' : 'mean', 
                            'biomass_var' : 'mean',
                            'harvest_index_var' : 'mean',
                            'canopy_cover_var': 'mean',
                            'Es_var' : 'mean',
                            'Tr_var': 'mean',
                            'DeepPerc_var': 'mean',
                            'CR_var': 'mean'})
group_var.reset_index(drop = False,inplace = True)

In [58]:
baseline.columns

Index(['Unnamed: 0', 'index', 'Unnamed: 0.1', 'run', 'Yield (tonne/ha)_mean',
       'Yield (tonne/ha)_min', 'Yield (tonne/ha)_max',
       'Yield (tonne/ha)_median', 'Yield (tonne/ha)_var', 'biomass_mean',
       'biomass_min', 'biomass_max', 'biomass_median', 'biomass_var',
       'harvest_index_mean', 'harvest_index_min', 'harvest_index_max',
       'harvest_index_median', 'harvest_index_var', 'canopy_cover_mean',
       'canopy_cover_min', 'canopy_cover_max', 'canopy_cover_median',
       'canopy_cover_var', 'Es_mean', 'Es_min', 'Es_max', 'Es_median',
       'Es_var', 'Tr_mean', 'Tr_min', 'Tr_max', 'Tr_median', 'Tr_var',
       'DeepPerc_mean', 'DeepPerc_min', 'DeepPerc_max', 'DeepPerc_median',
       'DeepPerc_var', 'CR_mean', 'CR_min', 'CR_max', 'CR_median', 'CR_var',
       'date_1', 'date_2', 'irrig_1', 'irrig_2', 'irrigation_1',
       'irrigation_2', 'depth'],
      dtype='object')

In [115]:
# construct naive t statistic using Welch's method for each interval versus the baseline.
baseline= pd.read_csv(F"{datadir}/all_results_baseline.csv")
td_10 = baseline[baseline.depth==10].copy()
td_10["baseline_10"] = "baseline_10"
td_25 = baseline[baseline.depth==25].copy()
td_25["baseline_25"] = "baseline_25"
td10 = td_10[["baseline_10","Yield (tonne/ha)_mean"]]
td25 = td_25[["baseline_25","Yield (tonne/ha)_mean"]]

mylist = []
for i in [10,25]:
    for j in dates:
        a= merged[merged.depth==i]
        c = a[a.interval==j]
        d = c[["depth","interval","Yield (tonne/ha)_mean"]]
        if i == 10:
            b = ttest_ind(d['Yield (tonne/ha)_mean'], td10['Yield (tonne/ha)_mean'],equal_var=False)
            mylist.append([b,F'{j}-{i}'])
        elif i == 25:
            b = ttest_ind(d['Yield (tonne/ha)_mean'], td25['Yield (tonne/ha)_mean'],equal_var=False)
            mylist.append([b,F'{j}-{i}'])

            results = pd.DataFrame(mylist)
test_results = results.rename(columns={results.columns[0]: 'Test Statistic, P-Value',results.columns[1]:'interval - depth'})
test_results

,"Test Statistic, P-Value",interval - depth
0,"(-0.14925941492076303, 0.8815961150926866)",30_60-10
1,"(8.62479086905845, 5.117457198073287e-14)",30_80-10
2,"(4.96064601868736, 2.374247572817184e-05)",30_110-10
3,"(-0.7578618380009042, 0.4543729343556355)",60_110-10
4,"(-12.059137965747222, 1.1819907447533288e-21)",60_130-10
5,"(4.124258372749213, 0.00024489933513373427)",80_110-10
6,"(-3.7443355118195876, 0.00047294141523656903)",80_140-10
7,"(-3.562057437394212, 0.0012813501924429898)",110_140-10
8,"(-14.48499911518838, 3.4192615518971323e-33)",30_60-25
9,"(-3.7308079299140795, 0.0003522648534320398)",30_80-25


The rest of this code is generating the same charts over and over for each variable in question.

In [ ]:
sns.set_style('darkgrid')
fig, axs = plt.subplots(2,2,figsize=(12,12))
data_ = merged
data = data_[['biomass_mean','depth','interval']]
sns.scatterplot(data=data,x='interval',y='biomass_mean',hue='depth',ax=axs[0,0])
data = group_mean
sns.lineplot(data=data,x='interval',y='biomass_mean',hue='depth',ax=axs[0,0])
axs[0,0].set_title('Mean')
data = data_[['biomass_max','depth','interval']]
sns.scatterplot(data=data,x='interval',y='biomass_max',hue='depth',ax=axs[0,1])
data = group_max
sns.lineplot(data=data,x='interval',y='biomass_max',hue='depth',ax=axs[0,1])
axs[0,1].set_title('Maximum')
data = data_[['biomass_var','depth','interval']]
sns.scatterplot(data=data,x='interval',y='biomass_var',hue='depth',ax=axs[1,0])
data = group_var
sns.lineplot(data=data,x='interval',y='biomass_var',hue='depth',ax=axs[1,0])
axs[1,0].set_title('Variance')
data = data_[['biomass_min','depth','interval']]
sns.scatterplot(data=data,x='interval',y='biomass_min',hue='depth',ax=axs[1,1])
data = group_min
sns.lineplot(data=data,x='interval',y='biomass_min',hue='depth',ax=axs[1,1])
axs[1,1].set_title('Minimum')
plt.tight_layout()
plt.savefig(F"{figdir}/biomass_int")

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(12,12))
data_ = merged
data = data_[['Yield (tonne/ha)_mean','depth','interval']]
sns.scatterplot(data=data,x='interval',y='Yield (tonne/ha)_mean',hue='depth',ax=axs[0,0])
data = group_mean
sns.lineplot(data=data,x='interval',y='Yield (tonne/ha)_mean',hue='depth',ax=axs[0,0])
axs[0,0].set_title('Mean')
data = data_[['Yield (tonne/ha)_max','depth','interval']]
sns.scatterplot(data=data,x='interval',y='Yield (tonne/ha)_max',hue='depth',ax=axs[0,1])
data = group_max
sns.lineplot(data=data,x='interval',y='Yield (tonne/ha)_max',hue='depth',ax=axs[0,1])
axs[0,1].set_title('Maximum')
data = data_[['Yield (tonne/ha)_var','depth','interval']]
sns.scatterplot(data=data,x='interval',y='Yield (tonne/ha)_var',hue='depth',ax=axs[1,0])
data = group_var
sns.lineplot(data=data,x='interval',y='Yield (tonne/ha)_var',hue='depth',ax=axs[1,0])
axs[1,0].set_title('Variance')
data = data_[['Yield (tonne/ha)_min','depth','interval']]
sns.scatterplot(data=data,x='interval',y='Yield (tonne/ha)_min',hue='depth',ax=axs[1,1])
data = group_min
sns.lineplot(data=data,x='interval',y='Yield (tonne/ha)_min',hue='depth',ax=axs[1,1])
axs[1,1].set_title('Minimum')
plt.tight_layout()
plt.savefig(F"{figdir}/yield_int")

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(12,12))
data_ = merged
data = data_[['canopy_cover_mean','depth','interval']]
sns.scatterplot(data=data,x='interval',y='canopy_cover_mean',hue='depth',ax=axs[0,0])
data = group_mean
sns.lineplot(data=data,x='interval',y='canopy_cover_mean',hue='depth',ax=axs[0,0])
axs[0,0].set_title('Mean')
data = data_[['canopy_cover_max','depth','interval']]
sns.scatterplot(data=data,x='interval',y='canopy_cover_max',hue='depth',ax=axs[0,1])
data = group_max
sns.lineplot(data=data,x='interval',y='canopy_cover_max',hue='depth',ax=axs[0,1])
axs[0,1].set_title('Maximum')
data = data_[['canopy_cover_var','depth','interval']]
sns.scatterplot(data=data,x='interval',y='canopy_cover_var',hue='depth',ax=axs[1,0])
data = group_var
sns.lineplot(data=data,x='interval',y='canopy_cover_var',hue='depth',ax=axs[1,0])
axs[1,0].set_title('Variance')
data = data_[['canopy_cover_min','depth','interval']]
sns.scatterplot(data=data,x='interval',y='canopy_cover_min',hue='depth',ax=axs[1,1])
data = group_min
sns.lineplot(data=data,x='interval',y='canopy_cover_min',hue='depth',ax=axs[1,1])
axs[1,1].set_title('Minimum')
plt.tight_layout()
plt.savefig(F"{figdir}/canopy_cover_int")

In [ ]:
sns.set_style('darkgrid')
fig, axs = plt.subplots(2,2,figsize=(12,12))
data_ = merged
data = data_[['Es_mean','depth','interval']]
sns.scatterplot(data=data,x='interval',y='Es_mean',hue='depth',ax=axs[0,0])
data = group_mean
sns.lineplot(data=data,x='interval',y='Es_mean',hue='depth',ax=axs[0,0])
axs[0,0].set_title('Mean')
data = data_[['Es_max','depth','interval']]
sns.scatterplot(data=data,x='interval',y='Es_max',hue='depth',ax=axs[0,1])
data = group_max
sns.lineplot(data=data,x='interval',y='Es_max',hue='depth',ax=axs[0,1])
axs[0,1].set_title('Maxmium')
data = data_[['Es_var','depth','interval']]
sns.scatterplot(data=data,x='interval',y='Es_var',hue='depth',ax=axs[1,0])
data = group_var
sns.lineplot(data=data,x='interval',y='Es_var',hue='depth',ax=axs[1,0])
axs[1,0].set_title('Variance')
data = data_[['Es_min','depth','interval']]
sns.scatterplot(data=data,x='interval',y='Es_min',hue='depth',ax=axs[1,1])
data = group_min
sns.lineplot(data=data,x='interval',y='Es_min',hue='depth',ax=axs[1,1])
axs[1,1].set_title('Minimum')
plt.tight_layout()
plt.savefig(F"{figdir}/Es_int")

In [ ]:
sns.set_style('darkgrid')
fig, axs = plt.subplots(2,2,figsize=(12,12))
data_ = merged
data = data_[['Tr_mean','depth','interval']]
sns.scatterplot(data=data,x='interval',y='Tr_mean',hue='depth',ax=axs[0,0])
data = group_mean
sns.lineplot(data=data,x='interval',y='Tr_mean',hue='depth',ax=axs[0,0])
axs[0,0].set_title('Mean')
data = data_[['Tr_max','depth','interval']]
sns.scatterplot(data=data,x='interval',y='Tr_max',hue='depth',ax=axs[0,1])
data = group_max
sns.lineplot(data=data,x='interval',y='Tr_max',hue='depth',ax=axs[0,1])
axs[0,1].set_title('Maximum')
data = data_[['Tr_var','depth','interval']]
sns.scatterplot(data=data,x='interval',y='Tr_var',hue='depth',ax=axs[1,0])
data = group_var
sns.lineplot(data=data,x='interval',y='Tr_var',hue='depth',ax=axs[1,0])
axs[1,0].set_title('Variance')
data = data_[['Tr_min','depth','interval']]
sns.scatterplot(data=data,x='interval',y='Tr_min',hue='depth',ax=axs[1,1])
data = group_min
sns.lineplot(data=data,x='interval',y='Tr_min',hue='depth',ax=axs[1,1])
axs[1,1].set_title('Minimum')
plt.tight_layout()
plt.savefig(F"{figdir}/Tr_int")

In [ ]:
sns.set_style('darkgrid')
fig, axs = plt.subplots(2,2,figsize=(12,12))
data_ = merged
data = data_[['DeepPerc_mean','depth','interval']]
sns.scatterplot(data=data,x='interval',y='DeepPerc_mean',hue='depth',ax=axs[0,0])
data = group_mean
sns.lineplot(data=data,x='interval',y='DeepPerc_mean',hue='depth',ax=axs[0,0])
axs[0,0].set_title('Mean')
data = data_[['DeepPerc_max','depth','interval']]
sns.scatterplot(data=data,x='interval',y='DeepPerc_max',hue='depth',ax=axs[0,1])
data = group_max
sns.lineplot(data=data,x='interval',y='DeepPerc_max',hue='depth',ax=axs[0,1])
axs[0,1].set_title('Maximum')
data = data_[['DeepPerc_var','depth','interval']]
sns.scatterplot(data=data,x='interval',y='DeepPerc_var',hue='depth',ax=axs[1,0])
data = group_var
sns.lineplot(data=data,x='interval',y='DeepPerc_var',hue='depth',ax=axs[1,0])
axs[1,0].set_title('Variance')
data = data_[['DeepPerc_min','depth','interval']]
sns.scatterplot(data=data,x='interval',y='DeepPerc_min',hue='depth',ax=axs[1,1])
data = group_min
sns.lineplot(data=data,x='interval',y='DeepPerc_min',hue='depth',ax=axs[1,1])
axs[1,1].set_title('Minimum')
plt.tight_layout()
plt.savefig(F"{figdir}/DeepPerc_int")

In [ ]:
sns.set_style('darkgrid')
fig, axs = plt.subplots(2,2,figsize=(12,12))
data_ = merged
data = data_[['DeepPerc_mean','depth','interval']]
sns.scatterplot(data=data,x='interval',y='DeepPerc_mean',hue='depth',ax=axs[0,0])
data = group_mean
sns.lineplot(data=data,x='interval',y='DeepPerc_mean',hue='depth',ax=axs[0,0])
axs[0,0].set_title('Mean')
data = data_[['DeepPerc_max','depth','interval']]
sns.scatterplot(data=data,x='interval',y='DeepPerc_max',hue='depth',ax=axs[0,1])
data = group_max
sns.lineplot(data=data,x='interval',y='DeepPerc_max',hue='depth',ax=axs[0,1])
axs[0,1].set_title('Maximum')
data = data_[['DeepPerc_var','depth','interval']]
sns.scatterplot(data=data,x='interval',y='DeepPerc_var',hue='depth',ax=axs[1,0])
data = group_var
sns.lineplot(data=data,x='interval',y='DeepPerc_var',hue='depth',ax=axs[1,0])
axs[1,0].set_title('Variance')
data = data_[['DeepPerc_min','depth','interval']]
sns.scatterplot(data=data,x='interval',y='DeepPerc_min',hue='depth',ax=axs[1,1])
data = group_min
sns.lineplot(data=data,x='interval',y='DeepPerc_min',hue='depth',ax=axs[1,1])
axs[1,1].set_title('Minimum')
plt.tight_layout()
plt.savefig(F"{figdir}/DeepPerc_int")

In [ ]:
sns.set_style('darkgrid')
fig, axs = plt.subplots(2,2,figsize=(12,12))
data_ = merged
data = data_[['harvest_index_mean','depth','interval']]
sns.scatterplot(data=data,x='interval',y='harvest_index_mean',hue='depth',ax=axs[0,0])
data = group_mean
sns.lineplot(data=data,x='interval',y='harvest_index_mean',hue='depth',ax=axs[0,0])
axs[0,0].set_title('Mean')
data = data_[['harvest_index_max','depth','interval']]
sns.scatterplot(data=data,x='interval',y='harvest_index_max',hue='depth',ax=axs[0,1])
data = group_max
sns.lineplot(data=data,x='interval',y='harvest_index_max',hue='depth',ax=axs[0,1])
axs[0,1].set_title('Maximum')
data = data_[['harvest_index_var','depth','interval']]
sns.scatterplot(data=data,x='interval',y='harvest_index_var',hue='depth',ax=axs[1,0])
data = group_var
sns.lineplot(data=data,x='interval',y='harvest_index_var',hue='depth',ax=axs[1,0])
axs[1,0].set_title('Variance')
data = data_[['harvest_index_min','depth','interval']]
sns.scatterplot(data=data,x='interval',y='harvest_index_min',hue='depth',ax=axs[1,1])
data = group_min
sns.lineplot(data=data,x='interval',y='harvest_index_min',hue='depth',ax=axs[1,1])
axs[1,1].set_title('Minimum')
plt.tight_layout()
plt.savefig(F"{figdir}/harvest_index_int")